In [ ]:
from math import exp, log, sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from linear_motion_sim import Simulator, fit_ks_and_kv, fit_ka, lognorm
from func_guesser import sign

In [ ]:
sim = Simulator(max_speed=20, max_accel=12, drag=0.8, noise=0.2)

In [ ]:
ks, kv = fit_ks_and_kv(sim)
ka = fit_ka(sim, ks, kv)

In [ ]:
ks *= lognorm(0.05)
kv *= lognorm(0.05)
ka *= lognorm(0.1)

In [ ]:
ac = sim.sim(lambda t, x, v, a: -ks + kv * v + ka * -2)
sim.plot_curves(ac)

In [ ]:
vtol = 0.1
xtol = 0.5
ttol = 0.1

In [ ]:
def ratelimit(x1, t1, x0, t0, v):
    dt = t1 - t0
    up = x0 + v * dt
    dn = x0 - v * dt
    return min(max(x1, dn), up)

In [ ]:
def smaller_mag(x, y):
    if abs(x) < abs(y):
        return x
    else:
        return y

In [ ]:
def clip_mag(x, max_mag):
    return x if abs(x) < max_mag else max_mag * sign(x)

\begin{align}
a&=\text{constant}\\
v(t) &= at + v_0\\
x(t) &= \tfrac 12 at^2 +v_0t
\end{align}

Solve for the missing boundary condition $v_0$ given the others.
\begin{align}
    x(0) &= 0\\
    x(t_d) &= d\\
    v(t_d) &= \text{constant}
\end{align}
Eliminate $t_d$
\begin{align}
    0 &= \tfrac 12 at_d^2 +v_0t_d-d \\
    t_d &= \frac{-v_0 \pm \sqrt{v_0^2 + 2 a d}}{a}
\end{align}
Also,
\begin{align}
    t_d &= \frac{v_d - v_0}{a}\\
    v_d - v_0 &= -v_0 \pm \sqrt{v_0^2 + 2 a d}\\
    v_d^2 &= v_0^2 + 2 a d\\
    v_0 &= \sqrt{v_d^2 - 2 a d}
\end{align}


In [ ]:
class Drive:
    def __init__(self,  
                 tgt_x,
                 tgt_v,
                 alimit_frac, 
                 vlimit):
        if sign(tgt_x) != sign(tgt_v):
            tgt_v = 0

        if alimit_frac < 0 or 1 < alimit_frac:
            raise Exception('alimit_frac should be in [0,1]')

        self.tgt_x = tgt_x
        self.tgt_v = tgt_v
        self.alimit = alimit_frac / ka
        self.vlimit = vlimit
        self.done_t = -1
        self.prev_t = 0


    def drive_power(self, t, x, v, a):
        if self.done_t >= 0:
            return 0

        if abs(x - self.tgt_x) < xtol and abs(v - self.tgt_v) < vtol:
            self.done_t = t
            return 0
        
        dist = self.tgt_x - x
        s = sign(dist)
        a0 = -s * self.alimit

        vd = self.tgt_v
        z = vd**2 - 2 * a0 * dist
        if z > 0:
            v0 = s * sqrt(z)
        else:
            # should not happen
            v0 = s * self.vlimit
        
        v1 = clip_mag(v0, self.vlimit)

        dt = t - self.prev_t
        
        next_a = clip_mag((v1 - v) / dt / 1.2, self.alimit)
 
        p = ks * s + kv * v + ka * next_a

        self.prev_t = t

        return p

In [ ]:
d = Drive(tgt_x=30, tgt_v=4, alimit_frac=0.8, vlimit=12)
ac = sim.sim(d.drive_power, max_t=10, v_init=0)
sim.plot_curves(ac, tgt_x=d.tgt_x)

In [ ]:
t, x, v, a, p = ac
i = t.searchsorted(d.done_t)
print(f'{x[i]} {v[i]}')